In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp 
import seaborn as sns
import itertools
import scipy.stats as ssp
%matplotlib inline
import seaborn as sns


import math
import random
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind

In [2]:
# Z BAZY DANYCH ZOSTAŁ WYRZUCONY 1993 R.

In [3]:
df=pd.read_csv('globalterrorismdb_0718dist.csv',encoding='ISO-8859-1')

C:\Users\48500\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df2=df[['iyear','imonth','iday','country_txt','region_txt','city','attacktype1_txt','targtype1_txt','weaptype1_txt','nkill','natlty1_txt']]

In [6]:
df3 = df2.dropna()

In [7]:
df3

,iyear,imonth,iday,country_txt,region_txt,city,attacktype1_txt,targtype1_txt,weaptype1_txt,nkill,natlty1_txt
0,1970,7,2,Dominican Republic,Central America & Caribbean,Santo Domingo,Assassination,Private Citizens & Property,Unknown,1.0,Dominican Republic
1,1970,0,0,Mexico,North America,Mexico city,Hostage Taking (Kidnapping),Government (Diplomatic),Unknown,0.0,Belgium
2,1970,1,0,Philippines,Southeast Asia,Unknown,Assassination,Journalists & Media,Unknown,1.0,United States
5,1970,1,1,United States,North America,Cairo,Armed Assault,Police,Firearms,0.0,United States
6,1970,1,2,Uruguay,South America,Montevideo,Assassination,Police,Firearms,0.0,Uruguay
...,...,...,...,...,...,...,...,...,...,...,...
181686,2017,12,31,Somalia,Sub-Saharan Africa,Ceelka Geelow,Armed Assault,Military,Firearms,1.0,Somalia
181687,2017,12,31,Syria,Middle East & North Africa,Jableh,Bombing/Explosion,Military,Explosives,2.0,Russia
181688,2017,12,31,Philippines,Southeast Asia,Kubentog,Facility/Infrastructure Attack,Private Citizens & Property,Incendiary,0.0,Philippines
181689,2017,12,31,India,South Asia,Imphal,Bombing/Explosion,Government (General),Explosives,0.0,India


In [73]:
# dodanie kolumny z indexem
df3['index'] = df3.index

C:\Users\48500\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [123]:
mc = df3.groupby(['nkill', 'region_txt']).count()['index'].reset_index()

In [124]:
mc.groupby('region_txt').mean()


,nkill,index
region_txt,,
Australasia & Oceania,6.200000,27.500000
Central America & Caribbean,61.430108,86.053763
Central Asia,15.739130,24.217391
East Asia,23.083333,31.458333
Eastern Europe,54.471698,93.132075
Middle East & North Africa,117.720588,352.558824
North America,143.259259,124.185185
South America,43.257143,246.371429
South Asia,70.607143,384.312500


In [ ]:
# czy średnia liczba ofiar z podziałem na rodzaj ataku

In [125]:
a = 0.05
x = np.array(mc.loc[mc['region_txt'] == 'Australasia & Oceania' ]['index'])
y = np.array(mc.loc[mc['region_txt'] == 'Central America & Caribbean']['index'])


print(ttest_ind(x,y))

Ttest_indResult(statistic=-0.39702463186274245, pvalue=0.6921870170807547)


In [ ]:
# WNIOSKI no. 1
# P - value jest większe od poziomu istotności, więc hipotezę zerową można przyjąć za prawdziwą.

In [110]:
def test_bootstrap(x,y,n=1000,alpha=0.05):
    pvalues= []
    for i in range(n):
        # losujemy ze zwracaniem wektory
        sample_x = np.random.choice(x,len(x),replace=True)
        sample_y = np.random.choice(y,len(y),replace=True)
        #test t-studenta
        tst = ttest_ind(sample_x,sample_y)
        # czy wynik jest istotny
        p = (tst.pvalue < alpha)*1
        pvalues.append(p)
        
    print('Wynik testu istotny w {0} symulacji'.format(  "{:.1%}".format(np.mean(pvalues) )))

In [111]:
test_bootstrap(x,y)

Wynik testu istotny w 0.5% symulacji


In [ ]:
# WNIOSKI no. 2
# Wynikiem jest 0,5% przypadków, jest to wyniki bliski 0, więc absolutnie liczba ofiar w Australasia & Oceania nie jest wyższa 
# od liczby ofiar w Central America & Caribbean

In [114]:
def test_symulacja(x,y,xm,y_m,xs,ys,n=1000,alpha=0.05):
    #xm,xs - średnia i odchylenie rozkałdu normalnego charakteryzująca różnicę ofiar dla regionu 'Australasia & Oceania'
    #ym,ys - to samo dla regionu 'Central America & Caribbean'
    pvalues= []
    for i in range(n):
        #dodajemy wylosowane różnice
        sample_x = x + np.random.normal(xm,xs,len(x)).round()
        sample_y = y + np.random.normal(y_m,ys,len(y)).round()
        tst = ttest_ind(sample_x,sample_y)
        p = (tst.pvalue < alpha)*1
        pvalues.append(p)
    print('Wynik testu istotny w {0} symulacji'.format(  "{:.1%}".format(np.mean(pvalues) )))

In [116]:

test_symulacja(x,y,0,100,500,1000)

Wynik testu istotny w 0.1% symulacji
